In [6]:
import os

os.environ['NVIDIA_API_KEY'] = "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"
from omegaconf import OmegaConf
import sys
sys.path.append("../")
sys.path.append("../../../")
from retriever_evalset_generator import RetrieverEvalSetGenerator
from filters import EasinessFilter, AnswerabilityFilter
from nemo_curator.modules.filter import ScoreFilter, Score
from nemo_curator.datasets import DocumentDataset
import warnings
warnings.filterwarnings('ignore')


/home/vinay/.local/lib/python3.10/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [7]:
import pandas as pd
df = pd.read_json("../data/nq_test.jsonl", lines=True)
df = df[:5]

In [8]:
cfg = OmegaConf.load("../config/retriever_eval_sdg.yaml")
retrieval_evalset_generator = RetrieverEvalSetGenerator(cfg)
retrieval_evalset_generator.init_pipeline_params()

In [9]:
dataset = DocumentDataset.from_pandas(df)
generated_dataset = retrieval_evalset_generator.run(dataset)

In [10]:
generated_dataset.df.head(n=5, npartitions=-1)

,_id,text,title,question,answer
0,doc6,"The fourth season of Chicago Fire, an American...",Chicago Fire (season 4),Who are the executive producer and producers o...,The executive producer of the fourth season of...
1,doc6,"The fourth season of Chicago Fire, an American...",Chicago Fire (season 4),When did the fourth season of Chicago Fire pre...,The fourth season of Chicago Fire premiered on...
2,doc6,"The fourth season of Chicago Fire, an American...",Chicago Fire (season 4),How many episodes were there in the fourth sea...,There were 23 episodes in the fourth season of...
3,doc10,"""Love Will Keep Us Alive"" is a song written by...",Love Will Keep Us Alive,"Who are the songwriters behind ""Love Will Keep...","The songwriters behind ""Love Will Keep Us Aliv..."
4,doc10,"""Love Will Keep Us Alive"" is a song written by...",Love Will Keep Us Alive,"Who produced the song ""Love Will Keep Us Alive""?","The song ""Love Will Keep Us Alive"" was produce..."


In [11]:
ef = EasinessFilter(cfg)
easiness_filter = ScoreFilter(ef,
                              text_field = ["text", "question"],
                              score_field = "scores")

In [12]:
filtered_dataset = easiness_filter(generated_dataset)
filtered_dataset.df.head(n=5, npartitions=-1)

,_id,text,title,question,answer,scores
0,doc6,"The fourth season of Chicago Fire, an American...",Chicago Fire (season 4),Who are the executive producer and producers o...,The executive producer of the fourth season of...,0.621422
3,doc10,"""Love Will Keep Us Alive"" is a song written by...",Love Will Keep Us Alive,"Who are the songwriters behind ""Love Will Keep...","The songwriters behind ""Love Will Keep Us Aliv...",0.580546
4,doc10,"""Love Will Keep Us Alive"" is a song written by...",Love Will Keep Us Alive,"Who produced the song ""Love Will Keep Us Alive""?","The song ""Love Will Keep Us Alive"" was produce...",0.576173
6,doc42,"""Fishin' in the Dark"" is a song written by Wen...",Fishin' in the Dark,"Who wrote the song ""Fishin' in the Dark""?","The song ""Fishin' in the Dark"" was written by ...",0.645659
7,doc42,"""Fishin' in the Dark"" is a song written by Wen...",Fishin' in the Dark,"When was the song ""Fishin' in the Dark"" releas...","The song ""Fishin' in the Dark"" was released as...",0.647324
